In [1]:
# 1. Install Required Libraries
!pip install tensorflow matplotlib opencv-python scikit-learn

# 2. Import Libraries
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix

C:\Users\abdul\anaconda3\envs\plantdoc\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
import zipfile
import os

zip_path = "PlantVillage.zip"  # change if the file name is different
extract_path = "PlantVillage"

# Create target folder if not exists
os.makedirs(extract_path, exist_ok=True)

# Extract zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"Dataset extracted to '{extract_path}/' folder")

Dataset extracted to 'PlantVillage/' folder


In [3]:
# 3. Dataset Path and Image Preprocessing

dataset_path = "PlantVillage/PlantVillage"  # Make sure this points to the folder with class subfolders
img_size = 128
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_data = datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.


In [4]:
# 4. Build CNN Model

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_data.num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

C:\Users\abdul\anaconda3\envs\plantdoc\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │         1,935 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,306,575 (12.61 MB)

 Trainable params: 3,306,575 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
#5. Train the Model
history = model.fit(train_data, validation_data=val_data, epochs=10)

C:\Users\abdul\anaconda3\envs\plantdoc\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 231s 445ms/step - accuracy: 0.3408 - loss: 2.0462 - val_accuracy: 0.6924 - val_loss: 0.9671
Epoch 2/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 117s 227ms/step - accuracy: 0.6722 - loss: 1.0029 - val_accuracy: 0.7196 - val_loss: 0.8193
Epoch 3/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 117s 226ms/step - accuracy: 0.7347 - loss: 0.7850 - val_accuracy: 0.8149 - val_loss: 0.5390
Epoch 4/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 113s 218ms/step - accuracy: 0.7662 - loss: 0.7003 - val_accuracy: 0.8506 - val_loss: 0.4485
Epoch 5/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 114s 220ms/step - accuracy: 0.8096 - loss: 0.5560 - val_accuracy: 0.8666 - val_loss: 0.3927
Epoch 6/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 114s 220ms/step - accuracy: 0.8204 - loss: 0.5259 - val_accuracy: 0.8920 - val_loss: 0.3123
Epoch 7/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 110s 214ms/step - accuracy: 0.8439 - loss: 0.4681 - val_accuracy: 0.9044 - val_loss: 0.2838
Epoch 8/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 109s 211ms/step - accuracy: 0.8611 -

In [6]:
# 7. Save the Trained Model (.h5)
model.save("leaf_disease_model.h5")

In [7]:
# 📲 8. Convert Model to TensorFlow Lite (.tflite) for Mobile

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('leaf_disease_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model saved as 'leaf_disease_model.tflite'")

INFO:tensorflow:Assets written to: C:\Users\abdul\AppData\Local\Temp\tmp70dmqclb\assets


INFO:tensorflow:Assets written to: C:\Users\abdul\AppData\Local\Temp\tmp70dmqclb\assets


Saved artifact at 'C:\Users\abdul\AppData\Local\Temp\tmp70dmqclb'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 15), dtype=tf.float32, name=None)
Captures:
  1933843541504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933843363616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933842642368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933842642192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933842644304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933842643952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933842695792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933842695616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933842697728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1933842697376: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model 